# The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2018 Semester 1
-----
## Project 1: What is labelled data worth to Naive Bayes?
-----
###### Student Name(s): Emmanuel Macario
###### Python version: 3.6.0

This iPython notebook is a template which you may use for your Project 1 submission. (You are not required to use it; in particular, there is no need to use iPython if you do not like it.)

Marking will be applied on the seven functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 

In [72]:
# Import useful libraries
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [73]:
# Filename constants used for easy file access
CSV1 = 'breast-cancer-dos.csv'
CSV2 = 'car-dos.csv'
CSV3 = 'hypothyroid-dos.csv'
CSV4 = 'mushroom-dos.csv'


def preprocess(filename):
    """
    Opens a data file in csv, and transforms it into
    a usable format.
    """
    
    # Read csv data into a dataframe. Assign
    # an integer value to each attribute in
    # the data.
    df = pd.read_csv(filename, header=None)
    
    
    # Partition the data into instances and
    # instance class labels.
    instance_list = df.iloc[:,:-1]
    class_list = df.iloc[:,-1]
    data_set = instance_list, class_list
    
    # Note: the instance list is a dataframe, whereas the class
    # list is a series object
    
    return data_set


# Test preprocess function
instance_list, class_list = preprocess(CSV2)
instance_list.head(5)

,0,1,2,3,4,5
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [74]:

def train_supervised(instance_list, class_list):
    """
    Builds a supervised Naive Bays model,
    given a preprocessed set of training
    data, by calculating counts from the
    training data.
    
    Inputs: a training set of data, consisting
    of a list of instances, and a list of class 
    labels for those instances.
    
    Outputs: 2-tuple containing a class frequency 
    dictionary and also the supervised model.
    """
        
    # The supervised Naive Bays model
    supervised_model = {}
    
    # For every instance in the training
    # data, update the model
    for i in range(len(instance_list)):
        
        # Get the instance
        instance = instance_list.iloc[i,:]
        
        # Get the associated class label
        class_label = class_list[i]
        
        
        # If the class label is not already in
        # the model, create a nested set of
        # dictionaries for the class.
        if class_label not in supervised_model:
            
            # New dictionary for every new class
            supervised_model[class_label] = {}
            
            # For each attribute, initialise
            # a new default dictionary.
            for attr in instance_list.columns:
                supervised_model[class_label][attr] = defaultdict(int)
                
        
        # For every attribute in the instance,
        # get its corresponding value and update
        # the model.
        for attr in instance_list.columns:
            
            attr_value = instance[attr]
            
            # If a value is missing in a training instance, it is
            # possible to simply have it not contribute to the
            # attribute-value counts.
            if attr_value == '?':
                print("Encountered missing value in:")
                print(instance)
                print()
                continue
                
            supervised_model[class_label][attr][attr_value] += 1
    
    
    return supervised_model



# Test function
supervised_model = train_supervised(instance_list, class_list)

# supervised_model

In [75]:

def get_class_freqs(supervised_model):
    """
    Returns a dictionary containing frequency counts
    of all the class labels in the training data.
    """
    
    class_freqs = defaultdict(int)
    
    for class_label in supervised_model:
        class_freqs[class_label] = sum(supervised_model[class_label][0].values())
        
    return class_freqs



def predict_supervised(supervised_model, instance_list):
    """
    Predicts the class labels for a set of test
    instances, based on a supervised Naive Bayes
    model.
    """
    
    # The list of predictions for each instance
    prediction_list = []
    
    # Get the class frequencies to avoid redundant calculations
    class_freqs = get_class_freqs(supervised_model)
    
    # Calculate the total number of instances
    total_instances = sum(class_freqs.values())
    
    
    # For each instance in the test set, predict 
    # its class based on the supervised model.
    for instance in instance_list.values:
        
        max_prob = 0.0
        predicted_class = ""
        
        # We obtain the probability of an instance
        # being a certain class, for each possible class.
        for class_label in class_freqs:
            
            # Firstly, get the prior probability of a class
            prob = class_freqs[class_label]/total_instances
            
            
            # Now, multiply the prior probability of the class
            # by the posterior probability of each of the instance's 
            # attribute values, given the class.
            for attr in supervised_model[class_label]:
                prob *= (supervised_model[class_label][attr][instance[attr]]/class_freqs[class_label])
                
            
            # If the probability is the highest seen
            # thus far, set the predicted class to the
            # class label.
            if prob > max_prob:
                max_prob = prob
                predicted_class = class_label
            
        
        # Add the class label with the highest
        # corresponding probability to the list
        # of predictions.
        prediction_list.append(predicted_class)
        
        
    return prediction_list


# Test the function
prediction_list = predict_supervised(supervised_model, instance_list)

In [76]:

def evaluate_supervised(prediction_list, class_list):
    """
    Evaluates a set of predictions, in a supervised
    context. Uses accuracy as the primary method of
    evaluation.
    """
    
    # Validation checking
    assert(len(prediction_list) == len(class_list))
    
    # Calculate and return the accuracy of the model
    return (prediction_list == class_list).value_counts().loc[True]/len(prediction_list)


# Test the function
evaluate_supervised(prediction_list, class_list)


0.8738425925925926

In [89]:

def train_unsupervised(instance_list, class_labels):
    """
    Builds a weak unsupervised Naive Bayes model,
    from a given set of unlabelled training data,
    and a unique set of possible class labels.
    """
    
    # The list containing random class
    # distributions for each instance.
    random_class_distributions = []
    
    
    # For each training instance, create
    # a non-uniform class distribution for
    # that instance.
    for instance in instance_list.values:
        
        random_distribution = np.random.rand(1,len(class_labels))[0]
        norm_random_distribution = random_distribution/sum(random_distribution)
                
        random_class_distributions.append(norm_random_distribution)
        
        #random_class_distributions.append(
        #    np.random.dirichlet(np.ones(len(class_labels)),size=1)[0])
    
    
    # Make a new dataframe consisting of random 
    # class distributions for each instance.
    class_distributions = pd.DataFrame(random_class_distributions, 
                                       columns=class_labels)   
        
        
    
    # Dictionary contains the (random) frequencies
    # of each class in the training set
    class_freqs = defaultdict(int) 
    
    
    # Count the frequencies
    for class_label in class_labels:
        class_freqs[class_label] = class_distributions[class_label].sum()
    
    
    # Initialise the unsupervised Naive Bays model, which
    # is just a dictionary of dictionaries of dictionaries
    unsupervised_model = {c : {a : defaultdict(float) for a in instance_list.columns} for c in class_labels}
    
    
    # Concatenate the dataframes
    # class_distributions = pd.concat([instance_list, class_distributions], axis=1)
    

    # For every instance in the training
    # data, update the unsupervised model
    for i in range(len(instance_list)):
        
        for attr in instance_list.columns:
            
            for class_label in class_labels:
                
                unsupervised_model[class_label][attr][instance_list[attr][i]] += class_distributions[class_label][i]
                
    
    return class_distributions, class_freqs, unsupervised_model



# Test the function
class_labels = class_list.unique()
cd, cf, um = train_unsupervised(instance_list, class_labels)
print([cd[class_label].sum() for class_label in cd.columns])

[441.44460132557424, 430.98196104778697, 423.16176457007856, 432.411673056559]


In [86]:
np.random.choice([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 4)

array([0.1, 0.2, 0.7, 0.2])

In [90]:
def get_class_freqs2(class_distributions):
    
    class_freqs = defaultdict(float)
    
    for class_label in class_distributions.columns:
        class_freqs[class_label] = class_distributions[class_label].sum()
        
    return class_freqs



def predict_unsupervised(class_distributions, unsupervised_model, instance_list, iterations=5):
    """
    Predicts the class distribution for a set of
    instances, based on a trained model.
    """
        
    # For each instance in the test set, iteratively
    # update the class distribution for the instance
    for iteration in range(iterations):
        
        # Calculate the frequency of the classes 
        # in the random class distribution
        class_freqs = get_class_freqs2(class_distributions)
        
        print(class_freqs)
        
        
        # Calculate the total number of instances
        total_instances = int(sum(class_freqs.values()) + 0.5)
    
        
        for i in range(len(instance_list)):

            # Get the instance
            instance = instance_list.iloc[i,:]

            # Find a new class distribution for the instance,
            # then normalise that distribution.
            for class_label in class_freqs:

                # Firstly, get the prior probability of a class
                prob = class_freqs[class_label]/total_instances

                # Now, multiply the prior probability of the class
                # by the posterior probability of each of the instance's 
                # attribute values, given the class.
                for attr in unsupervised_model[class_label]:
                    prob *= (unsupervised_model[class_label][attr][instance[attr]]/class_freqs[class_label])


                # Update the probability distribution for the instance
                class_distributions[class_label][i] = prob


        # Now, normalise the class distribution so that the probabilities add to 1
        class_distributions = class_distributions.div(class_distributions.sum(axis=1), axis=0)


    return class_distributions


# Test the function
class_distributions = predict_unsupervised(cd, um, instance_list)
class_distributions.head(25)

defaultdict(<class 'float'>, {'unacc': 441.44460132557424, 'acc': 430.98196104778697, 'vgood': 423.16176457007856, 'good': 432.411673056559})
defaultdict(<class 'float'>, {'unacc': 441.44498397667195, 'acc': 430.98196826795345, 'vgood': 423.1612052092658, 'good': 432.4118425461086})
defaultdict(<class 'float'>, {'unacc': 441.44307148542924, 'acc': 430.9819331594464, 'vgood': 423.1639986290006, 'good': 432.4109967261228})
defaultdict(<class 'float'>, {'unacc': 441.45263013409317, 'acc': 430.98210366667473, 'vgood': 423.1500485657512, 'good': 432.4152176334812})
defaultdict(<class 'float'>, {'unacc': 441.4048562373443, 'acc': 430.98127414011077, 'vgood': 423.21971724804365, 'good': 432.39415237449845})


,unacc,acc,vgood,good
0,0.254414,0.236040,0.260961,0.248586
1,0.251334,0.231768,0.274180,0.242718
2,0.252064,0.235402,0.272668,0.239866
3,0.261456,0.225852,0.261420,0.251271
4,0.258276,0.221752,0.274647,0.245325
5,0.259071,0.225267,0.273179,0.242484
6,0.265436,0.230046,0.242573,0.261945
7,0.262556,0.226170,0.255186,0.256088
8,0.263348,0.229741,0.253806,0.253105
9,0.253968,0.230042,0.273655,0.242336


In [91]:

def evaluate_unsupervised(class_distributions, class_list):
    """
    Evaluates a set of predictions, in an
    unsupervised manner.
    """
    
    # Validation checking
    assert(len(class_distributions) == len(class_list))
    
    # The list of class predictions for the instances
    prediction_list = []
        
    # Get the predictions for all instances
    for instance in class_distributions.values:
        prediction_list.append(class_distributions.columns[np.argmax(instance)])
    
    
    # Validation checking again
    assert(len(prediction_list) == len(class_list))
    
    # Construct a confusion matrix to calculate accuracy
    confusion_matrix = {predicted_class : defaultdict(int) for predicted_class in class_list.unique()}
    
    # Fill in the confusion matrix
    for i in range(len(prediction_list)):
        predicted_class = prediction_list[i]
        actual_class = class_list[i]
        confusion_matrix[predicted_class][actual_class] += 1
    
    
    # Calculate the total number of 'correct' predictions
    correct = 0
    for predicted_class in confusion_matrix:
        correct += max(confusion_matrix[predicted_class].values())
    
    
    print(confusion_matrix)
    
    # Calculate and return the accuracy of the classifier
    return correct/len(prediction_list)


# Test the function
evaluate_unsupervised(class_distributions, class_list)

{'unacc': defaultdict(<class 'int'>, {'unacc': 392, 'acc': 145, 'vgood': 22, 'good': 2}), 'acc': defaultdict(<class 'int'>, {'unacc': 260, 'acc': 82, 'vgood': 33, 'good': 45}), 'vgood': defaultdict(<class 'int'>, {'unacc': 250, 'acc': 108, 'good': 14, 'vgood': 6}), 'good': defaultdict(<class 'int'>, {'unacc': 308, 'acc': 49, 'good': 8, 'vgood': 4})}


0.7002314814814815

Questions (you may respond in a cell or cells below):

1. Since we’re starting off with random guesses, it might be surprising that the unsupervised NB works at all. Explain what characteristics of the data cause it to work pretty well (say, within 10% Accuracy of the supervised NB) most of the time; also, explain why it utterly fails sometimes.
2. When evaluating supervised NB across the four different datasets, you will observe some variation in effectiveness (e.g. Accuracy). Explain what causes this variation. Describe and explain any particularly suprising results.
3. Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out (hint: check out numpy.shuffle()) or cross–validation evaluation strategy. How does your estimate of Accuracy change, compared to testing on the training data? Explain why. (The result might surprise you!)
4. Implement one of the advanced smoothing regimes (add-k, Good-Turing). Do you notice any variation in the predictions made by either the supervised or unsupervised NB classifiers? Explain why, or why not.
5. The lecture suggests that deterministically labelling the instances in the initialisation phase of the unsupervised NB classifier “doesn’t work very well”. Confirm this for yourself, and then demonstrate why.
6. Rather than evaluating the unsupervised NB classifier by assigning a class deterministically, instead calculate how far away the probabilistic estimate of the true class is from 1 (where we would be certain of the correct class), and take the average over the instances. Does this performance estimate change, as we alter the number of iterations in the method? Explain why.
7. Explore what causes the unsupervised NB classifier to converge: what proportion of instances change their prediction from the random assignment, to the first iteration? From the first to the second? What is the latest iteration where you observe a prediction change? Make some conjecture(s) as to what is occurring here.

Don't forget that groups of 1 student should respond to question (1), and one other question. Groups of 2 students should respond to question (1), and three other questions. Your responses should be about 100-200 words each.